# Trace Generation and Modeling

To test our DeepValidate approach we generate a dataset of test traces from a chain of relatively simple arithmetical functions operating on a series of randomized inputs. Given the generated program traces, we train a LSTM classifier to predict whether the output will be valid or result in an error. 

The trace generation is performed by `output_trace.jl` which reproduces much of the functionality of `varextract.jl` with some important differences. Rather than send trace information to `stdout`, we direct the traces to a file `traces.dat`. 

(It must be noted that this is not possible within an IJulia notebook due to restrictions on [task switching in staged functions](https://github.com/JuliaLang/julia/issues/18568) which prevents the trace outputs from being written to a file recursively. However, this works just fine from the command line.)

In [ ]:
function Cassette.overdub(ctx::TraceCtx,
                          f,
                          args...)
    open("traces.dat", "a") do file
        write(file, string(f))
        write(file, string(args))
    end
    
    # if we are supposed to descend, we call Cassette.recurse
    if Cassette.canrecurse(ctx, f, args...)
        subtrace = (Any[],Any[])
        push!(ctx.metadata[1], (f, args) => subtrace)
        newctx = Cassette.similarcontext(ctx, metadata = subtrace)
        retval = Cassette.recurse(newctx, f, args...)
        # push!(ctx.metadata[2], subtrace[2])
    else
        retval = Cassette.fallback(ctx, f, args...)
        push!(ctx.metadata[1], :t)
        push!(ctx.metadata[2], retval)
    end
    @info "returning"
    @show retval
    return retval
end

We then modify our `@textset` so that it creates the `traces.dat` file and then loops through a large number of randomized runs of our arithmetic tests. Error conditions happen most often when our inputs are sufficiently close to zero, so a Normal(0,2) distribution gives us a good range of values to generate a reasonable percentage of "bad" traces on which to train. Empirically the share of "bad" traces generated is about 15-17%.

In [ ]:
@testset "TraceExtract" begin
    g(x) = begin
        y = add(x.*x, -x)
        z = 1
        v = y .- z
        s = sum(v)
        return s
    end
    h(x) = begin
        z = g(x)
        zed = sqrt(z)
        return zed
    end

    open("traces.dat", "w") do f
        write(f, "")
    end

    seeds = rand(Normal(0,2),30000,3)
    
    for i=1:size(seeds,1)
        ctx = TraceCtx(pass=ExtractPass, metadata = (Any[], Any[]))
        try
            result = Cassette.overdub(ctx, h, seeds[i,:])
        catch DomainError
            dump(ctx.metadata)
        finally
            open("traces.dat", "a") do f
                write(f, "\n")
            end
        end
        if i%1000 == 0
            @info string(i)
        end
    end
end


After generating our raw traces, a small amount of pre-processing is required before attempting to model around them. First, we classify our "good" and "bad" traces based on whether they have resulted in an error. 

We then need to strip out the actual error dump information from our "bad" traces, as this would too easily give away the prediction game. All traces end just before they would error, allowing the validation model to predict the that next outcome. 

In [ ]:
text = split(String(read("traces.dat")), "\n");
Ys = Int.(occursin.(Ref(r"(Base[\S(?!\))]+error)"i), text));

text = split.(text, Ref(r"(Base[\S(?!\))]+error)"i));
text = [t[1] for t in text];

sum(Ys)

Finally, we save our traces and our labels our as CSV files for easy ingestion for our model. 

In [ ]:
writedlm( "traces.csv",  text[1:end-1], ',')
writedlm( "y_results.csv",  Ys[1:end-1], ',')

## Validation Classifier Model
For our modeling, we use [Flux.jl](https://github.com/FluxML/Flux.jl) and train an LSTM encoder/decoder classifier on our traces.

In [3]:
using Pkg

In [4]:
#Pkg.add(["Flux", "MLDataPattern", "DelimitedFiles"])
Pkg.activate("../../")
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [5]:
using DelimitedFiles
using Flux
using Flux: onehot, throttle, crossentropy, onehotbatch, params, shuffle
using MLDataPattern: stratifiedobs
using Base.Iterators: partition

include("../../src/validation/utils.jl")


┌ Info: Recompiling stale cache file /home/jfairbanks6/.julia/compiled/v1.0/Flux/QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /home/jfairbanks6/.julia/compiled/v1.0/MLDataPattern/qdIQj.ji for MLDataPattern [9920b226-0b2a-5f5f-9153-9aa70a013f8b]
└ @ Base loading.jl:1190


get_data (generic function with 1 method)

In [6]:
#
# Set up inputs for model
#

# Read lines from traces.dat text in to arrays of characters
# Convert to onehot matrices

cd(@__DIR__)

text, alphabet, N = get_data("traces.dat")
stop = onehot('\n', alphabet);
prod(alphabet)

"}(*, [-0.34912857])ad6+Bseroctbmilznpy{DfuASqvF<_Mhwx:g\"#\n"

In [7]:
# Partition into subsequences to input to our model

seq_len = 50

Xs = [collect(partition(t,seq_len)) for t in text];

In [8]:
prod.(Xs[7])

31-element Array{String,1}:
 "getfield(Main, Symbol(\"#h#9\")){getfield(Main, Symb"  
 "ol(\"#g#8\"))}(getfield(Main, Symbol(\"#g#8\"))())([1."
 "2772, -3.93049, -1.6268],)getfield(getfield(Main, "    
 "Symbol(\"#h#9\")){getfield(Main, Symbol(\"#g#8\"))}(ge"
 "tfield(Main, Symbol(\"#g#8\"))()), :g)getfield(Main,"  
 " Symbol(\"#g#8\"))()([1.2772, -3.93049, -1.6268],)Ba"  
 "se.Broadcast.broadcasted(*, [1.2772, -3.93049, -1."    
 "6268], [1.2772, -3.93049, -1.6268])Base.Broadcast."    
 "materialize(Base.Broadcast.Broadcasted(*, ([1.2772"    
 ", -3.93049, -1.6268], [1.2772, -3.93049, -1.6268])"    
 "),)Base.Broadcast.instantiate(Base.Broadcast.Broad"    
 "casted(*, ([1.2772, -3.93049, -1.6268], [1.2772, -"    
 "3.93049, -1.6268])),)copy(Base.Broadcast.Broadcast"    
 ⋮                                                       
 "2, 4.27327], 1)Base.Broadcast.materialize(Base.Bro"    
 "adcast.Broadcasted(-, ([0.354036, 19.3792, 4.27327"    
 "], 1)),)Base.Broadcast.instantiate(Base.Br

In [9]:
Ys = (map(t->occursin("sqrt_llvm", prod(t)), text));

In [10]:
sum(Ys), length(Ys)

(297, 348)

In [11]:
Xs_vec = [[onehotbatch(x, alphabet) for x in Xs[i]] for i in 1:length(Xs)];
Xs_vec[1]

19-element Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1}:
 [true true … false false; false false … false false; … ; false false … false false; false false … false false]  
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false

In [12]:
#Ys = readdlm("y_results.csv");
labelset = unique(Ys)
#dataset = [(onehotbatch(x, alphabet, '\n'), onehot(Ys[i],labelset))
#           for i in 1:length(Ys) for x in Xs[i]] |> shuffle
dataset = shuffle([(Xs_vec[i], onehot(Ys[i], labelset)) for i in 1:length(Xs_vec)])
Ys = last.(dataset)
first.(dataset)[1][1:25]

25-element Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1}:
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [true false … false false; false true … true false; … ; false false … false false; false false … false false]   
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false

In [71]:
# Pad sequences to equal lengths
#Xs_padded = [hcat(x,repeat(stop,1,seq_len-size(x)[1])) for x in first.(dataset)]
Xs_padded = [hcat(x[end-10:end]...) for x in first.(dataset)]
#map(length, Xs_padded)

348-element Array{Array{Bool,2},1}:
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false fal

In [72]:
# There are 972,290 items in our data. We use a train:test split of 90:10, stratified to ensure we have 
# the same share of "bad" and "good" traces in our train and test sets.

(Xtrain, Ytrain), (Xtest, Ytest) = stratifiedobs((Xs_padded, Ys), p=0.9)

train = [(Xtrain[i], Ytrain[i]) for i in 1:length(Ytrain)];
test = [(Xtest[i], Ytest[i]) for i in 1:length(Ytest)];
length(train), length(test)

(313, 35)

In [73]:
train[1][1]

58×513 Array{Bool,2}:
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false   true  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
     ⋮                       

In [134]:
# We set up our model architecture

scanner = Chain(Dense(length(alphabet), seq_len, σ), GRU(seq_len, seq_len))
encoder = Dense(seq_len, 2)

function model(x)
    state = scanner.([x])[end]
  Flux.reset!(scanner)
  softmax(encoder(state))
end

loss(tup...) = begin
    #@show typeof.(tup)
    #@show size.(tup)
    crossentropy(model(tup[1]), tup[2])
end
accuracy(tup...) = mean(argmax(model(tup[1])) .== argmax(tup[2]))

#opt = ADAM(0.000001)
opt = NADAM()
ps = params(scanner, encoder);
#ps = params(model);

In [135]:
# Finally, we set up our callbacks for reporting on training progress.
mean(x) = sum(x)/length(x)
#testacc() = mean(accuracy(t) for t in test)
testloss() = mean(loss(t...) for t in test)

evalcb = () -> @show testloss()#, testacc()


#106 (generic function with 1 method)

In [136]:
# Now, train!
@show length(train), length(test)
epochs = 5
for e in 1:epochs
    Flux.train!(loss, ps, train, opt, cb = throttle(evalcb, 10))
end

(length(train), length(test)) = (313, 35)
testloss() = 355.30643f0 (tracked)
testloss() = 219.49481f0 (tracked)
testloss() = 218.9892f0 (tracked)
testloss() = 218.71275f0 (tracked)
testloss() = 218.54019f0 (tracked)


In [137]:
ps

Params([Float32[0.0046493 0.143189 … 0.0386976 -0.0220846; 0.0953533 -0.181361 … 0.0775406 0.159622; … ; -0.152562 0.0973889 … 0.0198131 0.226378; 0.192318 -0.202472 … 0.150506 0.0259108] (tracked), Float32[-0.0211291, 0.00633388, -0.0177386, 0.00857473, -0.0145447, 0.00963244, -0.0351977, -0.0236635, -0.0140145, -0.0215261  …  -0.0197102, 0.00941888, -0.0139004, 0.0076977, 0.00751663, 0.00264625, 0.00858244, 0.00838083, 0.00687756, 0.00899159] (tracked), Float32[-0.179534 -0.0860613 … -0.0136944 -0.0313459; 0.162765 -0.0957109 … 0.137274 0.0294737; … ; -0.136724 0.151115 … 0.0734992 0.0715915; -0.118607 -0.145648 … -0.106321 0.0516434] (tracked), Float32[-0.0821717 -0.0600535 … 0.172035 -0.165278; -0.0547273 0.0413193 … 0.0228902 -0.0649927; … ; -0.168567 -0.141629 … 0.0192246 0.144573; 0.034644 0.127264 … -0.0724915 0.104429] (tracked), Float32[0.10685, 0.106475, -0.004917, -0.141753, 0.166372, 0.0635171, -0.113672, 0.0779668, 0.167566, -0.182035  …  0.0963428, -0.138044, 0.104255, 0

In [138]:
#model.(Xs_padded)[:,end]

In [139]:
sumη = [0.0,0.0]
df = []
for i in 1:300
    yhat = model(train[i][1])[:,end]
    η = crossentropy(yhat, train[i][2])
    sumη += η.*train[i][2]
    υ = Flux.onecold(train[i][2])
    logods = round(Flux.data(yhat[1]/yhat[2]);digits=3)
    push!(df, (logods, υ, round(Flux.data(η);digits=5)))
    #println("$υ\t$logods\t$η")
end
ηbar = sumη./sum(train[i][2] for i in 1:length(train))


Tracked 2-element Array{Float64,1}:
 1.6958653771358987 
 0.17123413967729062

In [140]:
df[1:50]

50-element Array{Any,1}:
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 ⋮                      
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 

In [141]:
df[end-25:end]

26-element Array{Any,1}:
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 1, 1.81418f0)
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 
 (0.195f0, 2, 0.1779f0) 

In [40]:
Pkg.add(["CuArrays","CUDAnative"])

 Resolving package versions...
  Updating `~/.julia/dev/SemanticModels/Project.toml`
  [be33ccc6] + CUDAnative v1.0.1
  Updating `~/.julia/dev/SemanticModels/Manifest.toml`
 [no changes]


In [41]:
Pkg.build("CUDAnative")

  Building LLVM ──────→ `~/.julia/packages/LLVM/tPWXv/deps/build.log`
  Building CUDAdrv ───→ `~/.julia/packages/CUDAdrv/JWljj/deps/build.log`
  Building CUDAnative → `~/.julia/packages/CUDAnative/Mdd3w/deps/build.log`


┌ Error: Error building `CUDAnative`: 
│ ERROR: LoadError: CUDA toolkit at  doesn't contain nvcc
│ Stacktrace:
│  [1] error(::String) at ./error.jl:33
│  [2] find_toolkit_version(::Array{String,1}) at /home/jfairbanks6/.julia/packages/CUDAapi/AVyQs/src/discovery.jl:277
│  [3] main() at /home/jfairbanks6/.julia/packages/CUDAnative/Mdd3w/deps/build.jl:125
│  [4] top-level scope at none:0
│  [5] include at ./boot.jl:317 [inlined]
│  [6] include_relative(::Module, ::String) at ./loading.jl:1044
│  [7] include(::Module, ::String) at ./sysimg.jl:29
│  [8] include(::String) at ./client.jl:392
│  [9] top-level scope at none:0
│ in expression starting at /home/jfairbanks6/.julia/packages/CUDAnative/Mdd3w/deps/build.jl:165
└ @ Pkg.Operations /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Pkg/src/Operations.jl:1097


In [103]:
prod([alphabet[i] for i in Flux.onecold(Xs_padded[17])])

"t.instantiate(Base.Broadcast.Broadcasted(-, ([1.66631, -0.0850698, 1.40313], 1)),)copy(Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{1}}(-, ([1.66631, -0.0850698, 1.40313], 1)),)sum([0.666307, -1.08507, 0.403127],)sqrt(-0.01563633837697509,)zero(-0.01563633837697509,)oftype(-0.01563633837697509, 0)typeof(-0.01563633837697509,)convert(Float64, 0)Float64(0,)sitofp(Float64, 0)<(-0.01563633837697509, 0.0)lt_float(-0.01563633837697509, 0.0)Base.Math.throw_complex_domainerror(:sqrt, -0.01563633837697509)"

In [108]:
model(Xs_padded[17])[1,end-55:end]

Tracked 56-element Array{Float32,1}:
 0.19566849f0
 0.16220814f0
 0.16472563f0
 0.19566849f0
 0.16500941f0
 0.16945647f0
 0.19585176f0
 0.16734757f0
 0.1622731f0 
 0.16945647f0
 0.17063263f0
 0.16502388f0
 0.16387472f0
 ⋮           
 0.16331194f0
 0.16331194f0
 0.16506495f0
 0.16331194f0
 0.15888843f0
 0.15918225f0
 0.16096874f0
 0.15888843f0
 0.16172567f0
 0.17136525f0
 0.16096874f0
 0.16254273f0

In [128]:
testset = first.(test)
trainset = first.(train)
len = length(alphabet)
mauto = Chain(LSTM(len, len),
    Dense(len, 10),
    Dense(10, len),
    LSTM(len, len)
)
aeloss(x) = crossentropy(x, mauto(x))
ps = params(mauto);
testloss() = mean(aeloss.(test))
aeevalcb = () -> @show testloss()#, testacc()


#102 (generic function with 1 method)

In [131]:
mauto(trainset[1])

Tracked 58×513 Array{Float32,2}:
 -0.0705769  -0.0666339   -0.0732878   …  -0.0705769  -0.0647261  -0.0664111 
 -0.0805568  -0.0703508   -0.0748242      -0.0805568  -0.0839093  -0.0749225 
  0.0138432   0.0185871    0.0173942       0.0138432   0.0164735   0.0186596 
 -0.0681567  -0.072485    -0.0628791      -0.0681567  -1.06836    -0.0659251 
  0.077986    0.0731708    0.0727787       0.077986    0.0678245   0.0600028 
 -0.0579838  -0.054669    -0.0517083   …  -0.0579838  -0.0557668  -0.0533445 
  0.016597    0.00993134   0.0222545       0.016597    0.015693    0.011876  
  0.0255617   0.0282435    0.0352516       0.0255617   0.0277976   0.0339066 
 -0.0602138  -0.0542954   -0.0621746      -0.0602138  -0.0641329  -0.0621729 
  0.0502398   0.0484903    0.0477855       0.0502398   0.0621099   0.0542963 
 -1.07647    -0.0759812   -0.0678374   …  -1.07647    -0.0694822  -0.0605956 
 -0.0542272  -1.05185     -0.0518987      -0.0542272  -0.0544686  -0.0505509 
 -0.080803   -0.0802201   -0.08

In [132]:
aeloss(trainset[1])

NaN (tracked)

In [129]:
epochs = 5
for e in 1:epochs
    Flux.train!(aeloss, ps, trainset, opt, cb = throttle(aeevalcb, 10))
end

MethodError: MethodError: no method matching aeloss(::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bool, ::Bo